In [1]:
import numpy as np
from gwpy.frequencyseries import FrequencySeries

# 1. 軸の定義（あえて不揃いにする）
# 入力軸: 低周波、細かい分解能 (0-100Hz, df=0.1Hz) -> 1000点
f_in = np.linspace(0, 100, 1000)

# 出力軸: 高周波、粗い分解能 (100-500Hz, df=1Hz) -> 400点
f_out = np.linspace(100, 500, 400)

# 2. 応答関数マップの作成 (shape: 400 x 1000)
# 例: 50Hzの環境ノイズが、200Hz付近の重力波チャンネルに結合する
coupling_matrix = np.zeros((len(f_out), len(f_in)))

# インデックスを見つけて値をセット
idx_in_50hz = np.argmin(np.abs(f_in - 50.0))
idx_out_200hz_start = np.argmin(np.abs(f_out - 190.0))
idx_out_200hz_end = np.argmin(np.abs(f_out - 210.0))

# 結合係数をセット (50Hz入力 -> 190-210Hz出力 に分布)
coupling_matrix[idx_out_200hz_start:idx_out_200hz_end, idx_in_50hz] = 1e-21

# BifrequencyMap インスタンス化
transfer_map = BifrequencyMap.from_points(
    coupling_matrix, 
    f_out=f_out, 
    f_in=f_in,
    name="Up-conversion Map",
    unit="strain/T"
)

# 3. 入力データの用意（軸がマップと少しずれていてもOK）
# 実際のデータは df=0.25Hz で取れているとする
f_measured = np.linspace(0, 100, 400) 
measured_noise = FrequencySeries(
    np.random.rand(400) + 10.0, # ダミーの磁場スペクトル
    frequencies=f_measured,
    unit="T",
    name="Magnetometer"
)

# 4. 伝搬計算 (自動補間が働く)
# マップの入力軸(0.1Hz刻み)に合わせて measured_noise(0.25Hz刻み) が補間され、計算される
projected_gw_noise = transfer_map.propagate(measured_noise)

print(projected_gw_noise)
# <FrequencySeries([0., 0., ..., 0.]
#                  unit=Unit("strain"),
#                  name='Projected: Up-conversion Map x Magnetometer',
#                  epoch=None,
#                  f0=100.0 Hz,   <-- 出力軸の開始周波数になっている
#                  df=1.0025 Hz)> <-- 出力軸の分解能になっている

/home/washimi/miniforge3/envs/ws-base/lib/python3.12/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


NameError: name 'BifrequencyMap' is not defined